# Toronto portal data

link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M <br>
Toronto - 103 FSAs

<b>Import packages</b>

In [146]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
import pandas as pd
import numpy as np
import re

<b>Download wiki page</b>

In [9]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(wiki_url) 

<b>Parser html file by BeautifulSoup (bs4)</b>

In [15]:
soup = BeautifulSoup(r.text, 'html.parser')  # parser html

In [16]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XowNfgpAICEAAJb@VM8AAAAI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":949497198,"wgRevisionId":949497198,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [21]:
table = soup.find('table')    #find the first table
print(table.tbody.contents)

In [99]:
columns = [h.text.strip() for h in table.tbody.contents[0].children if (type(h) == Tag)]   #Get header
table_df = pd.DataFrame(columns = columns)

In [110]:
#parser and add each row tr to dataframe
for tr in table.tbody.contents[1:]:
    if type(tr) == Tag:
        tr_seri = [td.text.strip() for td in tr.find_all('td')]
        print(tr_seri)
        table_df.loc[len(table_df)] = tr_seri


['M1A', 'Not assigned', '']
['M2A', 'Not assigned', '']
['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront']
['M6A', 'North York', 'Lawrence Manor / Lawrence Heights']
['M7A', 'Downtown Toronto', "Queen's Park / Ontario Provincial Government"]
['M8A', 'Not assigned', '']
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Malvern / Rouge']
['M2B', 'Not assigned', '']
['M3B', 'North York', 'Don Mills']
['M4B', 'East York', 'Parkview Hill / Woodbine Gardens']
['M5B', 'Downtown Toronto', 'Garden District, Ryerson']
['M6B', 'North York', 'Glencairn']
['M7B', 'Not assigned', '']
['M8B', 'Not assigned', '']
['M9B', 'Etobicoke', 'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale']
['M1C', 'Scarborough', 'Rouge Hill / Port Union / Highland Creek']
['M2C', 'Not assigned', '']
['M3C', 'North York', 'Don Mills']
['M4C', 'East York', 'Woodbine Heights']
['M5C', 'Downtown Toronto'

<b>Handle data</b>

In [112]:
#rename header
table_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
table_df

In [118]:
#drop Not assigned Borough
table_df = table_df[table_df['Borough'] != 'Not assigned'].reset_index(drop = True)

In [132]:
#repalce / by , in Neighborhood
table_df['Neighborhood'] = list(map(lambda x: x.replace('/', ',') , table_df['Neighborhood']))

In [144]:
# add borough to Not assigned Neighborhood
table_df[table_df['Neighborhood'] == 'Not assigned']['Neighborhood'] == table_df[table_df['Neighborhood'] == 'Not assigned']['Borough']

Series([], dtype: bool)

In [148]:
table_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [149]:
table_df.to_csv('toronto_portal.csv')

In [150]:
table_df.shape

(103, 3)